In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [3]:
# Set up the Spotify API
client_id = 'your client id'
client_secret = 'your client secret'

In [4]:
# Authenticate with the Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [5]:
# Fetch playlist data
playlist_id = '2TcJW6fV7JKdd4aAiYRGYc' # use your own playlist id
results = sp.playlist_items(playlist_id, offset=0, limit=100)

In [6]:
# Initialize variables
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

In [8]:
# Extract and display playlist data
data = []  # Store track data
artist_ids = set()  # Store artist IDs

for item in tracks:
    track = item['track']
    artist_id = track['artists'][0]['id']
    artist_ids.add(artist_id)
    data.append({
        'name': track['name'],
        'artist': track['artists'][0]['name'],
        'artist_id': artist_id,
        'album': track['album']['name'],
        'duration': round(track['duration_ms'] / 60000, 2),  # Convert duration to minutes
        'release_date': track['album']['release_date'],
        'popularity': track['popularity'],
        'track_url': track['external_urls']['spotify'],
        'image_url': track['album']['images'][0]['url'] if track['album']['images'] else None  # Check if images list is not empty
    })

In [9]:
# Create a DataFrame
df = pd.DataFrame(data)

In [10]:
# Fetch artist genres
artist_genres={}

for artist_id in artist_ids:
    artist = sp.artist(artist_id)
    artist_genres[artist_id] = artist['genres']

In [11]:
# Add genres to the dataframe
df['genres'] = df['artist_id'].map(artist_genres)

# genre column extraction had to be done separately because the genre column was a list of lists and not a list of strings as expected by the DataFrame 

In [12]:
# Display the dataframe
print(df.head())

                                            name                  artist  \
0                                         Attack  Thirty Seconds To Mars   
1                                       The Kill  Thirty Seconds To Mars   
2                                           Grit                 palence   
3   Major Tom (Völlig losgelöst) - 2022 Remaster         Peter Schilling   
4  Major Tom (Völlig losgelöst) - Single Version         Peter Schilling   

                artist_id                                        album  \
0  0RqtSIYZmd4fiBKVFqyIqD                              A Beautiful Lie   
1  0RqtSIYZmd4fiBKVFqyIqD                              A Beautiful Lie   
2  2hP9ky7jNcqy6F3nYKSGt8                                         Grit   
3  7ip3CWlgPZbQHvgJpmcGSS             Fehler im System (2023 Remaster)   
4  7ip3CWlgPZbQHvgJpmcGSS  Major Tom (Völlig losgelöst) [All Versions]   

   duration release_date  popularity  \
0      3.14   2005-01-01          63   
1      3.86   2005

In [13]:
# Save to CSV
df.to_csv('1 - ScrapedPlaylist.csv', index=False)